In [1]:
import cv2
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
import os, tqdm
import pandas as pd
import seaborn as sns
from sklearn.svm import SVC

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import auc, roc_curve
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay
from collections import Counter
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import StandardScaler
# from sklearn.feature_extraction.image import extract_patches_2d

from PIL import Image

from skimage.feature import fisher_vector, learn_gmm

from typing import *

# import wandb
# os.environ["WANDB_ENTITY"] = "c3-mcv"
# wandb.login(key = '14a56ed86de5bf43e377d95d05458ca8f15f5017', relogin=True)

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Reshape, Input
def build_mlp(in_size, out_size, num_layers, activation, phase='train'):
    model = Sequential()
    model.add(Input(shape=(in_size, in_size, 3,), name='input'))
    model.add(Reshape((in_size*in_size*3,)))

    if in_size*in_size*3 < out_size: 
        #increment size
        size_step = (out_size - in_size*in_size*3) / (num_layers - 1)
        sign = 1
    else: 
        #decrement size 
        size_step = (in_size*in_size*3 - out_size) / (num_layers - 1)
        sign = -1

    # Add layers
    for i in range(num_layers - 1):
        layer_size = int(in_size*in_size*3 + sign * size_step * i)
        model.add(Dense(units=layer_size, activation=activation))

    model.add(Dense(units=out_size, activation=activation))

    # In the feature extractor phase, stop building the model before the last layer
    if phase == 'feature_extractor':
        return model
    
    else:
        model.add(Dense(units=8, activation='linear' if phase == 'test' else 'softmax'))
        return model

In [7]:
class MLP_BoVW():
    def __init__(self, config, size_per_class=1e9, data_path='./MIT_split', model_path = './models/mlp.h5'):
        """
        Bag-of-Visual-Words (BoVW) image classifier.

        Parameters:
        - config: Dictionary containing configuration parameters for the BoVW model.
        - size_per_class: Maximum number of images per class to use for training.
        - data_path: Path to the dataset folder.
        """
        self.config = config
        self.size_per_class = size_per_class
        self.data_path = data_path
        self._initialize_datasets()

        # Compute features for each split
        self.train_features = self._compute_features(self.train_dataset_blocks['image_paths'], 'train', model_path)
        self.test_features = self._compute_features(self.test_dataset_blocks['image_paths'], 'test', model_path)

        # Classification
        if self.config['classifier'] == 'knn':
            self.classifier = KNeighborsClassifier(n_neighbors=self.config['n_neigh'], n_jobs=-1, metric=self.config['metric'])
        elif self.config['classifier'] == 'svm':
            self.classifier = SVC(kernel = self.config['kernel'], degree=self.config['degree_pol'], class_weight = 'balanced', gamma = 'auto', C = self.config['C'], probability=True, random_state=123)
        elif self.config['classifier'] == 'logistic':
            self.classifier = LogisticRegression(multi_class = 'auto', penalty='l2', max_iter=300, solver='lbfgs', C = self.config['C'], class_weight = 'balanced', n_jobs=-1, random_state=123)
        
        # Dimensionality reduction
        self.dim_red = None
        if self.config['n_components'] > 0:
            self.dim_red = PCA(n_components = self.config['n_components'])

        # Standarization
        self.scaler = None
        if self.config['scaler']:
            self.scaler = StandardScaler(with_mean=True, with_std=True)
    
    def _compute_features(self, dataset, type_dataset, model_path):
        """
        Computes the features for the train and test splits using dense SIFT.
        """

        if self.config['features'] == 'mlp':
            self.mlp = build_mlp(in_size=self.config['patch_size'], out_size=self.config['out_size'], num_layers=self.config['num_layers'], activation='relu', phase='feature_extractor')
            # model.load_weights(model_path)
        elif self.config['features'] == 'dense_sift':
            # Initialize Dense SIFT extractor
            sift = cv2.SIFT_create()
            kp = [cv2.KeyPoint(x, y, self.config['kp_scale']) for y in range(0, self.config['patch_size'], self.config['step_size'])
                                                              for x in range(0, self.config['patch_size'], self.config['step_size'])]
        
        # Initialize features matrix
        n = self.n_images_train if type_dataset == 'train' else self.n_images_test
        n_features = len(kp) if self.config['features'] == 'dense_sift' else 1
        feat_dim = 128 if self.config['features'] == 'dense_sift' else self.config['out_size']
        features = np.empty((n, self.n_patches_per_image, n_features, feat_dim))

        idx_img = -1
        batch = []
        # Compute features for each image
        for j, filename in tqdm.tqdm(enumerate(dataset), desc=f'Extracting features from {type_dataset} dataset', total=len(dataset)):
            if j % self.n_patches_per_image == 0:
                idx_img += 1

            # Load image
            img = cv2.imread(filename)
            color = cv2.COLOR_BGR2GRAY if self.config['features'] == 'dense_sift' else cv2.COLOR_BGR2RGB
            img = cv2.cvtColor(img, color)
            
            # Compute descriptors
            if self.config['features'] == 'dense_sift':
                _, des = sift.compute(img, kp) # shape (n_kp, 128)
                features[idx_img, j % self.n_patches_per_image, :, :] = des
            elif self.config['features'] == 'mlp':
                batch.append(img)
            
        if self.config['features'] == 'mlp':
            batch = np.array(batch)
            des = self.mlp.predict(batch, verbose=1)
            des = des.reshape(n, self.n_patches_per_image, 1, -1)

        return features

    def _create_directory(self, path):
        if not os.path.exists(path):
            os.makedirs(path, exist_ok=True)
            return False
        return True

    def _extract_patches(self, image_path, save_path, dataset_blocks, steps, block_path_exists):
        """
        Splits an image into patches.

        :param image_path: Path to the input image.
        :param destination_path: Path where the patches will be saved.
        :param dataset_blocks: Dictionary containing the paths to the patches and their corresponding labels.
        :param steps: Number of steps to move the sliding window.
        """
        if block_path_exists:
            # get the paths that contains the image name in image path
            paths = [os.path.join(save_path, path) for path in os.listdir(save_path) if path.startswith(os.path.basename(image_path).split('.')[0]+'_')]
            dataset_blocks['image_paths'].extend(paths)
            dataset_blocks['labels'].extend([os.path.basename(save_path)] * len(paths))
            return dataset_blocks

        # Load the image
        image = Image.open(image_path)

        i = 0
        # Extract and save patches
        for x in steps:
            for y in steps:
                patch_path = os.path.join(save_path, f"{os.path.splitext(os.path.basename(image_path))[0]}_{i}.jpg")
                box = (x, y, x + self.config['patch_size'], y + self.config['patch_size'])
                image.crop(box).save(patch_path)
                dataset_blocks['image_paths'].append(patch_path)
                dataset_blocks['labels'].append(os.path.basename(save_path))
                i += 1
        
        return dataset_blocks

    def _process_split(self, split, block_path):
        dataset = {'image_paths': [], 'labels': []}
        dataset_blocks = {'image_paths': [], 'labels': []}
        split_path = os.path.join(self.data_path, split)

        # Calculate the number of patches
        steps = range(0, 256 - self.config['patch_size'] + 1, self.config['patch_size'] - self.config['patch_size']//self.config['overlap'])
        self.n_patches_per_image = len(steps)**2

        for label in tqdm.tqdm(os.listdir(split_path), desc=f'Creating {split} patches...'):
            label_path = os.path.join(split_path, label)
            block_path_exists = self._create_directory(os.path.join(block_path, split, label))

            for i, image_name in enumerate(os.listdir(label_path)):
                if i >= self.size_per_class:
                    break

                image_path = os.path.join(label_path, image_name)
                dataset['image_paths'].append(image_path)
                dataset['labels'].append(label)

                dataset_blocks = self._extract_patches(image_path, os.path.join(block_path, split, label), dataset_blocks, steps, block_path_exists)

        dataset['labels'] = np.array(dataset['labels'])
        return dataset, dataset_blocks

    def _initialize_datasets(self):
        block_path = self.data_path + f'_{self.config["patch_size"]}_blocks_{self.config["overlap"]}_overlap'
        _ = self._create_directory(block_path)
        self.train_dataset, self.train_dataset_blocks = self._process_split('train', block_path)
        self.test_dataset, self.test_dataset_blocks = self._process_split('test', block_path)

        self.n_images_train = len(self.train_dataset['image_paths'])
        self.n_images_test = len(self.test_dataset['image_paths'])

    def fit(self, train_features=None, y_train_labels=None):
        """
        Fit the Bag of Visual Words (BoVW) model using training data.

        Parameters:

        """
        
        if train_features is None and y_train_labels is None:
            train_features = self.train_features
            y_train_labels = self.train_dataset['labels']

        n, _, _, dim_des = train_features.shape[-1] #n_images, n_patches_per_image, n_features, feat_dim
            
        # Clustering for the visual words
        if self.config["fisher"]:
            self.cluster = learn_gmm(train_features.reshape(-1,dim_des), n_modes=self.config["n_words"], gm_args={'n_init': 1, 'max_iter': 50, 'covariance_type':'diag'})
        else:
            self.cluster = MiniBatchKMeans(n_clusters=self.config['n_words'], n_init='auto', compute_labels=False, random_state=123)
            self.cluster.fit(train_features.reshape(-1,dim_des))
        
        # Calculate the visual words for each image and level
        dim_size = self.config['n_words'] if not self.config["fisher"] else 2*self.config['n_words']*dim_des + self.config['n_words'] # n_words or 2KD+K for fisher
        visual_words_train = np.zeros((n, self.n_patches_per_image, dim_size), dtype=np.float64) # we will need shape (n_images, dim_size)

        for i in range(n):
            for patch in range(self.n_patches_per_image):
                words = self.cluster.predict(train_features[i,patch]) if not self.config["fisher"] else fisher_vector(train_features[i,patch], self.cluster)
                hist = np.bincount(words, minlength=self.config['n_words'])
                hist /= sum(hist)
                print(hist)
                visual_words_train[i,patch,:] = hist if not self.config["fisher"] else words

        visual_words_train = visual_words_train.reshape(n, -1) #get shape (n_images, dim_size)

        # Standarization
        if self.config['scaler']:
            visual_words_train = self.scaler.fit_transform(visual_words_train)

        # Dimensionality reduction
        if self.dim_red is not None:
            visual_words_train = self.dim_red.fit_transform(visual_words_train)
        
        # Compute distance/kenrel matrix
        if self.config['classifier'] == 'knn' and self.config['metric'] == 'precomputed':
            self.visual_words_train_old = visual_words_train.copy()
            visual_words_train = histogram_intersection_distance(visual_words_train, visual_words_train)
        elif self.config['classifier'] == 'svm' and self.config['kernel'] == 'precomputed':
            self.visual_words_train_old = visual_words_train.copy()
            visual_words_train = histogram_intersection_kernel(visual_words_train, visual_words_train)
            
        # Fit the classifier
        self.classifier.fit(visual_words_train, y_train_labels)

config = {
    'patch_size': 64,
    'overlap': 16, # overlap proportion of patches -> 4 means 1/4 of the patch is overlapped
    'step_size': 32, # distance between dense sift keypoints
    'kp_scale': 8, # size of dense sift keypoints
    'out_size': 2048, # size of the output of the mlp
    'num_layers': 3, # number of layers of the mlp
    'features': 'mlp',
    'classifier': 'logistic',
    'scaler': True,
    'n_components': 0,
    'n_neigh': 1,
    'metric': 'euclidean',
    'kernel': 'linear',
    'degree_pol': 3,
    'C': 1,
}
bovw = MLP_BoVW(config, data_path='./MIT_split')

Extracting features from train dataset: 100%|██████████| 30096/30096 [00:19<00:00, 1518.24it/s]


(30096, 64, 64, 3)
941/941 [==============================] - 296s 314ms/step
(30096, 2048)
(1881, 16, 1, 2048)


Extracting features from test dataset: 100%|██████████| 12912/12912 [00:07<00:00, 1692.00it/s]


(12912, 64, 64, 3)
404/404 [==============================] - 128s 317ms/step
(12912, 2048)
(807, 16, 1, 2048)


In [5]:
bovw.train_features.shape

(1881, 16, 4, 128)